In [2]:
import pandas as pd
import numpy as np

import api,utils

If it's not making sense - there's removed code at line 160 of api.py

In [24]:
competition_types = api.get_competition_types()
competition_types = competition_types[competition_types['Abbreviation'].isin(["WCH", "U23WCH", "JWCH"])]
venues = api.get_venues()['RegionCity']

def get_relevant_competitions(year:int)->pd.DataFrame:
    """ Inputs an year and returns a dataframe with ids and venues of any Regular, Junior and U23 World Championships from that year""" 

    competition_ids_df = api.get_competitions(year = year)
    competition_ids_df = competition_ids_df[competition_ids_df['competitionTypeId'].isin(competition_types.index)]
    competition_ids_df = competition_ids_df[['Year', 'venueId', 'competitionTypeId']].merge(competition_types['Abbreviation'], how= "left",left_on='competitionTypeId', right_index= True)

    competition_ids_df['City'] = competition_ids_df['venueId'].apply((lambda x: venues[x]))

    competition_ids_df = competition_ids_df[['Year', 'Abbreviation', 'City']]
    competition_ids_df.rename(columns={'Abbreviation':'CompetitionType'}, inplace=True)

    return competition_ids_df

In [25]:
competitions_df = pd.DataFrame()

for year in range(1980, 2025):
    competitions_df = pd.concat([competitions_df, get_relevant_competitions(year)])

display(competitions_df.tail(3), competitions_df.head(3))

,Year,CompetitionType,City
id,,,
32228428-db12-474e-8fdd-891f280a5ee0,2024,U23WCH,St. Catharines
63da80b7-58e3-413a-baae-c5e08b606d93,2024,WCH,St. Catharines
00d87130-478c-4a3f-ac13-6f770b8b9e04,2024,JWCH,St. Catharines


,Year,CompetitionType,City
id,,,
8695e858-ceeb-4b76-82f5-1acc24c47db5,1980,WCH,Hazewinkel
c3446ea8-92c3-4646-b47a-fe2e84bcee97,1980,JWCH,Hazewinkel
45a27735-6626-48f7-a50b-bc127e248238,1981,JWCH,Sofia


In [13]:
competitions_df.to_csv('competition_df.csv')

In [48]:
competitions_df=pd.read_csv('competition_df.csv', index_col='id')

In [51]:
def get_competitions_results(competition_id=None, finals_only=False, boat_classes=None):
    """
    Takes the competition id and boat classes to be considered. Can restrict scope to final rounds only
    Returns a dataframe with columns "Boat", "Time", "Country", "Rank", "Date", "RoundName"

    Adapted from api.get_pgmts to only do what we need"""
    competition_races = api.get_competition_races(competition_id)
    competition_events = api.get_competition_events(competition_id)
    competition_results = api.get_race_results(competition_id=competition_id)
    if boat_classes is None: boat_classes = api.get_boat_types()

    if competition_results.empty:
        return competition_results

    if finals_only:
        competition_races = competition_races.loc[
            competition_races.racePhaseId == api.RACE_PHASES["Final"]
        ]
    
    def convert_PhaseId(id):
        phase = 'Unknown'
        try: phase = {k:v for v,k in api.RACE_PHASES.items()}[id]
        finally: return phase
    competition_races['RacePhase'] = \
        competition_races['racePhaseId'].apply(convert_PhaseId)

    competition_results = utils.merge(
        (
            competition_results.reset_index(),
            competition_races[["eventId", "Date", "DisplayName", 'RacePhase']],
            competition_events["boatClassId"],
            boat_classes,
        ),
        left_on=("raceId", "eventId", "boatClassId"),
        right_on="id",
    )

    results = competition_results.loc[
        competition_results.ResultTime > pd.to_timedelta(0),
        [
            "DisplayName_y", #comes from boat_classes and gives the code of the boat class
            "ResultTime",
            "Country",
            "Rank",
            "Date",
            "RacePhase",
            "DisplayName_x" #comes from competition_races and refers to if it's Final A or B
        ],
    ]
    results.columns = ["Boat", "Time", "Country", "Rank", "Date","RacePhase", 'RoundName']
    results["Rank"] = results.Rank.fillna(0).astype(int) 
    results.Time = results.Time.dt.total_seconds().apply(utils.format_totalseconds)
    return results.sort_values(by=['Date','Boat','RoundName', 'Rank']).reset_index(drop=True)

While running the next cell, I had to ammend some of 'api.py' in order to bypass errors without solving them. I'll get them sorted after all is loaded.

In [52]:
from itertools import product as iter_product

def get_boat_codes(age = ["", "J", "B"], sex = ["M", "W"], boat_size = ["8+", "4+", "4-", "4x"]): 
    """ Prepares the codes of the boat classes we need
                e.g. U23 Men's coxed fours becomes BM4+ """
    boat_codes = []
    list_products = iter_product(age,sex,boat_size)
    for l in list_products:
        a,b,c=l
        boat_codes.append(a+b+c)
    return boat_codes

def get_boat_codes_df(boat_codes):
    """ Subsets the boat_types df from api to the boat classes in our scope """
    b = api.get_boat_types()
    b = b.loc[b['DisplayName'].isin(boat_codes)]
    return b

boat_codes=get_boat_codes()
boat_codes_df = get_boat_codes_df(boat_codes)
results = pd.DataFrame()

for index,row in competitions_df.iterrows():
    df = get_competitions_results(competition_id=index, boat_classes=boat_codes_df) #finals_only=True ?
    #print(row['Year'], row['CompetitionType'])
    
    if df.empty: continue
    df = df[['Boat', 'Time', 'Country', 'Rank','Date','RacePhase','RoundName']]
    df['Year'] = row['Year']
    df['City'] = row['City']
    results = pd.concat([results, df])

c:\Users\plang\OneDrive\Documents\Portfolio time bby\api.py:159: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates = pd.to_datetime(records[col], errors='ignore')
c:\Users\plang\OneDrive\Documents\Portfolio time bby\api.py:159: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates = pd.to_datetime(records[col], errors='ignore')
c:\Users\plang\OneDrive\Documents\Portfolio time bby\api.py:159: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dates = pd.to_datetime(records[col], errors='ignore')
c:\Users\plang\OneDrive\Documents\Portfolio time bby\api.py:159: UserWarning: Could n

ValueError: expected hh:mm:ss format before .

In [63]:
results.shape
results.head(3)

,Boat,Time,Country,Rank,Date,RacePhase,RoundName,Year,City
0,JM4+,4:45.80,BUL,1,1980-08-17 00:00:00,Final,FA,1980,Hazewinkel
1,JM4+,4:46.41,RFA,2,1980-08-17 00:00:00,Final,FA,1980,Hazewinkel
2,JM4+,4:46.91,RDA,3,1980-08-17 00:00:00,Final,FA,1980,Hazewinkel


The problems were:
1. Some ranks are missing (NaN). We replace them with 0 and then 
1. Some of the datetimes were missing and are noted down as 0001-01-01 00:00:00, which is beyond what python's datetime can handle. We simply don't parse the dates and leave them in string format

In [60]:
display(results[results['Rank']==0])
display(results[results['Date']=="0001-01-01 00:00:00"].value_counts('Year'))

,Boat,Time,Country,Rank,Date,RacePhase,RoundName,Year,City
114,M8+,5:37.68,USA,0,0001-01-01 00:00:00,Heat,Men's Eight Heat E2,1993,Racice
269,W4x,6:56.12,AUS,0,0001-01-01 00:00:00,Final,Women's Quadruple Sculls Final FC,1995,Tampere
270,W4x,7:03.15,NOR,0,0001-01-01 00:00:00,Final,Women's Quadruple Sculls Final FC,1995,Tampere
0,M4x,6:20.92,BEL,0,1998-09-09 00:00:00,Repechage,R1,1998,Cologne
94,JM8+,6:01.59,CAN,0,2004-07-28 00:00:00,Repechage,R,2004,Banyoles
128,JW4-,7:23.33,ITA,0,2004-07-29 00:00:00,Repechage,R,2004,Banyoles


Year
1995    340
1993    179
1994    147
1985     69
2011     10
Name: count, dtype: int64

In [61]:
results.to_csv('world_rowing_results_2.csv')